<a href="https://colab.research.google.com/github/piesauce/oreilly-llm-course/blob/main/Agent101_oreilly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent 101 (LangGraph) — Weather-Only Agent (Qwen)
Minimal tutorial notebook:
1) Qwen routes the query: **WEATHER** vs **OTHER**
2) If WEATHER: Qwen extracts the **place** → call **free** Open‑Meteo tools → Qwen writes the final answer in plain English using tool results
3) If OTHER: reply exactly **I am not equipped to handle this.**

Runs locally on an **A100** (no paid APIs).

In [1]:
!pip -q install -U langgraph langchain-core transformers accelerate sentencepiece requests folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.2/502.2 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
import json, requests
from typing import TypedDict, Optional, Dict, Any

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import folium

from langgraph.graph import StateGraph, END

In [3]:
# ---- Qwen model (bigger, A100-friendly) ----
# If you want even bigger later: Qwen/Qwen2.5-14B-Instruct (may require more VRAM).
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True,
)
model.eval()

def qwen_generate(messages, max_new_tokens=128):
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
        )
    text = tokenizer.decode(out[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True).strip()
    return text

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [4]:
# ---- LLM intent: WEATHER vs OTHER (strict) ----
def llm_classify_weather_or_other(user_query: str) -> str:
    messages = [
        {"role": "system", "content": "You are an intent classifier. Output EXACTLY one token: WEATHER or OTHER."},
        {"role": "user", "content":
            "Label WEATHER if the request is about weather/forecast/temperature/rain/snow/wind/humidity/umbrella.\n"
            "Otherwise label OTHER.\n\n"
            f"Query: {user_query}\nLabel:"}
    ]
    text = qwen_generate(messages, max_new_tokens=2)
    first = text.split()[0].upper() if text else "OTHER"
    if "WEATHER" in first:
        return "WEATHER"
    if "OTHER" in first:
        return "OTHER"
    return "OTHER"

In [5]:
# ---- LLM place extraction (strict JSON) ----
# Returns {"place": "<string>"} or {"place": null}
def llm_extract_place(user_query: str) -> Optional[str]:
    messages = [
        {"role": "system", "content":
            "Extract the place (city/region/country) from the user's query.\n"
            "Return ONLY valid JSON with exactly one key: place.\n"
            "If no place is mentioned, set place to null.\n"
            "Prefer geocoder-friendly names (e.g., 'Toronto, Canada')."},
        {"role": "user", "content":
            "Examples:\n"
            "Q: What's the weather in Toronto tomorrow?\nA: {\"place\":\"Toronto, Canada\"}\n"
            "Q: Do I need an umbrella in Minneapolist today?\nA: {\"place\":\"Minneapolis, United States\"}\n"
            "Q: Do I need an umbrella today?\nA: {\"place\":null}\n\n"
            f"Q: {user_query}\nA:"}
    ]
    text = qwen_generate(messages, max_new_tokens=60)

    try:
        obj = json.loads(text)
    except Exception:
        import re
        m = re.search(r"\{.*\}", text, flags=re.DOTALL)
        if not m:
            return None
        try:
            obj = json.loads(m.group(0))
        except Exception:
            return None

    place = obj.get("place", None)
    if place is None:
        return None
    place = str(place).strip()
    return place if place else None

In [6]:
# ---- Free "tools": Open‑Meteo Geocoding + Forecast (no API keys) ----
def geocode_city(place: str) -> Optional[Dict[str, Any]]:
    url = "https://geocoding-api.open-meteo.com/v1/search"
    r = requests.get(url, params={"name": place, "count": 1, "language": "en", "format": "json"}, timeout=20)
    r.raise_for_status()
    data = r.json()
    results = data.get("results") or []
    if not results:
        return None
    hit = results[0]
    return {
        "name": hit.get("name"),
        "admin1": hit.get("admin1"),
        "country": hit.get("country"),
        "lat": hit.get("latitude"),
        "lon": hit.get("longitude"),
        "timezone": hit.get("timezone", "auto"),
    }

def forecast_daily(lat: float, lon: float, days: int = 3) -> Dict[str, Any]:
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_probability_max,wind_speed_10m_max",
        "forecast_days": days,
        "timezone": "auto",
    }
    r = requests.get(url, params=params, timeout=20)
    r.raise_for_status()
    return r.json()

def simple_map(lat: float, lon: float, label: str):
    m = folium.Map(location=[lat, lon], zoom_start=10, control_scale=True)
    folium.Marker([lat, lon], tooltip=label, popup=label).add_to(m)
    return m

def normalize_place_for_geocoder(place: str) -> str:
    # lightweight normalization without changing the overall flow
    p = place.strip()
    p = p.replace("USA", "United States").replace("U.S.A.", "United States").replace("US", "United States")
    if "NEW YORK" in p.upper() and "UNITED STATES" in p.upper() and "CITY" not in p.upper():
        p = "New York City, New York, United States"
    return p

In [7]:
# ---- LLM final answer (plain English using tool outputs) ----
def llm_weather_answer(user_query: str, geo: Dict[str, Any], forecast: Dict[str, Any]) -> str:
    # Keep the model grounded: it must only use the provided JSON tool outputs.
    tool_blob = {
        "geocoding": geo,
        "forecast": {
            "timezone": forecast.get("timezone"),
            "daily_units": forecast.get("daily_units"),
            "daily": forecast.get("daily"),
        }
    }
    messages = [
        {"role": "system", "content":
            "You are a helpful weather assistant.\n"
            "Use ONLY the provided JSON tool data. Do not invent values.\n"
            "Answer in plain English, concise.\n"
            "If the user asks about 'tomorrow' or 'today', pick the relevant date from daily.time.\n"
            "If the question is broad, summarize the next 1–3 days.\n"
            "Mention temperatures, precipitation chance, and wind if available."},
        {"role": "user", "content":
            f"User query: {user_query}\n\n"
            f"Tool data (JSON):\n{json.dumps(tool_blob, ensure_ascii=False)}\n\n"
            "Write the answer:"}
    ]
    return qwen_generate(messages, max_new_tokens=220)

In [8]:
# ---- LangGraph agent ----
class AgentState(TypedDict):
    query: str
    intent: str
    place: str
    geo: Optional[Dict[str, Any]]
    forecast: Optional[Dict[str, Any]]
    response: str
    map_html: Optional[str]

def route_intent(state: AgentState) -> AgentState:
    state["intent"] = llm_classify_weather_or_other(state["query"])
    return state

def do_weather(state: AgentState) -> AgentState:
    place = llm_extract_place(state["query"]) or "Toronto, Ontario, Canada"
    place = normalize_place_for_geocoder(place)
    state["place"] = place

    geo = geocode_city(place)
    # small fallback if still missing (common alias issue)
    if (geo is None) and ("NEW YORK" in place.upper()):
        geo = geocode_city("New York City, New York, United States")
        state["place"] = "New York City, New York, United States"

    state["geo"] = geo
    if not geo:
        state["response"] = f"I couldn't find '{place}'. Try a different city."
        state["forecast"] = None
        state["map_html"] = None
        return state

    fc = forecast_daily(geo["lat"], geo["lon"], days=3)
    state["forecast"] = fc

    # Final plain-English answer from the LLM (grounded in tool JSON)
    state["response"] = llm_weather_answer(state["query"], geo, fc)

    # Map
    m = simple_map(geo["lat"], geo["lon"], geo["name"])
    state["map_html"] = m._repr_html_()
    return state

def do_refuse(state: AgentState) -> AgentState:
    state["response"] = "I am not equipped to handle this."
    state["place"] = ""
    state["geo"] = None
    state["forecast"] = None
    state["map_html"] = None
    return state

graph = StateGraph(AgentState)
graph.add_node("route", route_intent)
graph.add_node("weather", do_weather)
graph.add_node("refuse", do_refuse)

graph.set_entry_point("route")
graph.add_conditional_edges(
    "route",
    lambda s: "weather" if s["intent"] == "WEATHER" else "refuse",
    {"weather": "weather", "refuse": "refuse"},
)
graph.add_edge("weather", END)
graph.add_edge("refuse", END)

app = graph.compile()

def run_agent(user_query: str):
    state: AgentState = {
        "query": user_query,
        "intent": "",
        "place": "",
        "geo": None,
        "forecast": None,
        "response": "",
        "map_html": None,
    }
    out = app.invoke(state)
    print(out["response"])
    if out.get("map_html"):
        from IPython.display import HTML, display
        display(HTML(out["map_html"]))
    return out

In [10]:
# Try it
#run_agent("What's the weather in Toronto tomorrow? Show me a map.")
#run_agent("Do I need an umbrella in Minneapolis today?")
run_agent("Explain diffusion models in one paragraph.")

I am not equipped to handle this.


{'query': 'Explain diffusion models in one paragraph.',
 'intent': 'OTHER',
 'place': '',
 'geo': None,
 'forecast': None,
 'response': 'I am not equipped to handle this.',
 'map_html': None}